# Load data

In [1]:
import numpy as np 
from monty.json import MontyDecoder
from monty.serialization import loadfn

data = loadfn('bulk_moduli.json')
structures = data['structures']
targets = np.log10(data['bulk_moduli'])

# Set up model and train

In [2]:
from megnet.data.crystal import CrystalGraph
from megnet.data.graph import GaussianDistance
from megnet.models import MEGNetModel

model = MEGNetModel(10, 2, nblocks=1, lr=1e-2,
                    n1=4, n2=4, n3=4, npass=1, ntarget=1,
                    graph_converter=CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 10), 0.5)))

Using TensorFlow backend.


In [3]:
model.train(structures, targets, epochs=2)

Epoch 1/2
1/1 [==============================] - 3s 3s/step - loss: 3.9710
Epoch 2/2
1/1 [==============================] - 0s 52ms/step - loss: 3.7993


# Prediction

## Predict from structure

In [4]:
from pymatgen import MPRester
mpr = MPRester()

# let's get some structure
structure = mpr.get_structure_by_material_id('mp-1143')

In [5]:
predicted_K = 10 ** model.predict_structure(structure).ravel()
print('The predicted K for {} is {} GPa'.format(structure.formula, predicted_K[0]))

The predicted K for Al4 O6 is 1.2169744968414307 GPa
